# Baseball rules engine VI

May 24, 2024

Version V is my first pygame rules engine, which is text based. That got a bit heavy before I had all the functionality in place.
Spent May 21-24 re-designing this in other documents.

Now, want to implement the leanest possible foundation in Jupyter in order to layer on additional requirements.

In [165]:
import copy

## 1. Baserunner class

In [395]:
class Baserunner:
    
    def __init__(self, name, base):
        self.name = name
        
        self.occupied_base = base
        self.attained_base = base
        
        self.f2_base = None
        self.tagup_base = None

## 2. Apply and remove Forces
### Apply Forces
Triggered by user input
- Called by Main > change_state() > state_bip()
- Takes in: 
    - Baserunner object (the Batter)
        - occupied = attained = 0B
        - f2_base = 1B
    - The dict of BR's -- {name: Baserunner object}
- Starting from Batter, identifies Forces and applies to each BR object
- Returns the dict of runners -- runners may be changed 

In [391]:
class ApplyForces:

        
    def apply_forces(self, batter, runners):
        
        f2bases = {1: batter, 2: False, 3: False, 4: False}
        
        for base in [1, 2, 3]:
        
            for runner in runners.values():
                attained = runner.attained_base
            
                if base == attained and f2bases[base]:
                    f2bases[ base + 1 ] = runner
                    runner.f2_base = base + 1
                        
        self.print_output(runners)
        
        return runners

    
    def print_output(self, runners):
        print("In Apply Forces:")
        for name, runner in runners.items():
            #print(base, runner)
            if runner:
                print(f"  > {name} is on {runner.attained_base}B, and forced to {runner.f2_base}B")
                
        print()

### Remove Forces
Triggered by FBC or fielder tag (base or BR)
- Called by Main > ?(several paths)? > put_out()
- Takes in:
    - Baserunner object being put out
    - The dict of BR's -- {name: Baserunner object}
- Starting from base of runner being put out, removes Forces for all preceding BR (e.g., out = 1B, 2B = remove force)
- Returns the dict of runners -- runners may be changed 

In [378]:
class RemoveForces:
    
    def remove_forces(self, runner_out, runners):
        kill_force_base = runner_out.f2_base ## This is the base the 'runner_out' is forced to
        
        print("In Remove Forces:")
        
        for base in range(kill_force_base, 5): ## Make it iterate to 5 to avoid edge cases
            
            print(f"  > Attempting to remove force on base {base}.", end = " ... ")
            
            for runner in runners.values():

                if base == runner.f2_base:
                    runner.f2_base = None
                    print(f"Yes -- {runner.name}'s Force removed", end = "")
            print()
        
        print()
            
        return runners

### 2.5 Unit test for 1.0 and 2.0

In [393]:
def test_make_BR():
    isaac = Baserunner('Isaac', 2)
    jack = Baserunner('Jack', 1)
    batter = Baserunner('Mr. Batter', 0)
    
    batter.f2_base = 1

    return {'Isaac': isaac, 'Jack': jack, 'Mr. Batter': batter}


def unit_test_2():
    af = ApplyForces()
    rf = RemoveForces()
    
    runners = test_make_BR()
    batter = runners['Mr. Batter']
    
    ## Apply Forces
    runners = af.apply_forces(batter, runners)
    
    runner_out_name = 'Jack' ## <-- **** UPDATE HERE ****
    print(f"{runner_out_name} is Out\n")
    runner_out = runners[runner_out_name]
    
    ## Remove Forces
    runners = rf.remove_forces(runner_out, runners)
    
    af.print_output(runners)
    

unit_test_2()

In Apply Forces:
  > Isaac is on 2B, and forced to 3B
  > Jack is on 1B, and forced to 2B
  > Mr. Batter is on 0B, and forced to 1B

Jack is Out

In Remove Forces:
  > Attempting to remove force on base 2. ... Yes -- Jack's Force removed
  > Attempting to remove force on base 3. ... Yes -- Isaac's Force removed
  > Attempting to remove force on base 4. ... 

In Apply Forces:
  > Isaac is on 2B, and forced to NoneB
  > Jack is on 1B, and forced to NoneB
  > Mr. Batter is on 0B, and forced to 1B



## 3. Main

### User input
I think I just call user_input( [item 1, item 2... ])

- Runner tag out: *['tag r', 'isaac']* >>>>> len = 2 | str, str
- Base tag out: *['tag b', 1 or 2 or 3]* >>>>> len = 2 | str, int

- Occupy base: *['occupy', 'isaac', 1 or 2 or 3 or 4]* >>>>> len = 3 | str, str, int

- Change state: *['state', 0 or 1, '' or 'fbc' or 'bip']* >>>>> len = 3 | str, int, str

In [394]:
class RuEg:
    
    def __init__(self):
        self.names = ['Isaac,', 'Jack,', 'JD', 'Romo', 'Casey', 'Sam', 'Pasma', 'Bradey', 'Kemper', 'Liam']
        self.runners = {} # dict: {name: object}
        self.runners_out = []
        self.state = -1 
        self.af = ApplyForces()
        self.rf = RemoveForces()
        self.batter = None


    ### Commanded functions     
    def master_do(self, li):
        key = li[0]
                
        func_map = {'tag r': self.fielder_tags_runner, 'tag b': self.fielder_tags_base,
                   'occupy': self.occupy_attain_base, 'state': self.change_state}

        func_map[ key ]( li )  # Call the function associated with the key and pass the rest of the list.
        

    def change_state(self, li):
        new_state = li[1]
        
        if new_state == 0:
            print("State updated to 0: Pre-pitch")
            self.state = 0

        elif new_state == 1:
            self.state = 1
            
            self.create_batter() ## Instantiate a new runner at base 0
            
            for name, runner in self.runners.items():
                runner.occupied_base = None
            
            if li[2] == 'bip':
                self.state_bip()

            elif li[2] == 'fbc':
                self.state_fbc()

                
    def state_bip(self):
        print("State updated to 1: BIP")
        self.runners = self.af.apply_forces(self.batter, self.runners)
        

    def state_fbc(self):
        print("State updated to 1: FBC")
        #temp_runners = copy.deepcopy( self.runners )
        name_of_runner_out = None

        for name, runner in self.runners: #temp_runners.items():
            if runner.attained_base == 0:
                self.put_out(name)
                name_of_runner_out = name

            else: 
                runner.f2_base = runner.attained_base
         
        if name_of_runner_out:
            del self.runners[name_of_runner_out]

    
    def create_batter(self):
        self.batter = self.create_runner(0)
        self.batter.f2_base = 1
        
        
    def create_runner(self, base):
        name = self.names.pop(0)
        runner = Baserunner(name, base)
        self.runners[name] = runner
        
        return runner
    
    
    
    """!!! PUT OUT !!!"""

    def put_out(self, name):
        runner_out = self.runners[name]
        #self.runners_out.append(runner_out)
        self.names.append(name) # Recycle the name        
        self.runners = self.rf.remove_forces(runner_out, self.runners)


    
    """!!! UTILITIES !!!"""
    
    
    def print_status(self):
        
        for name, runner in self.runners.items():
            #print(f"hi")
            print(f" {name} on {runner.attained_base} occupies {runner.occupied_base}", end = " | ")
            print(f"He is forced to {runner.f2_base} or must tag-up at {runner.tagup_base}")

                  
    def print_options(self):
        print("Call ru.master_do() and pass a list:")
        print(f"  > Tag runner: ['tag r', name = str]")
        print(f"  > Tag base: ['tag b', base = int]")
        print(f"  > Runner occupy a base: ['occupy', name = str, base = int]")
        print(f"  > Change state: ['state', int, '' or 'fbc' or 'bip' = str]")
        print("\nCreate and place runner:")
        print(f"  > ru.create_runner(start_base = int)")

ru = RuEg()
ru.print_options()

Call ru.master_do() and pass a list:
  > Tag runner: ['tag r', name = str]
  > Tag base: ['tag b', base = int]
  > Runner occupy a base: ['occupy', name = str, base = int]
  > Change state: ['state', int, '' or 'fbc' or 'bip' = str]

Create and place runner:
  > ru.create_runner(start_base = int)


***

In [334]:
ru.create_runner(2)
#ru.print_status()

#print()

li = ['state', 0, '']
ru.master_do( li )

print()

ru.print_status()

State updated to 0: Pre-pitch

 Isaac, on 2 occupies 2 | He is forced to None or must tag-up at None
 Jack, on 0 occupies 0 | He is forced to 1 or must tag-up at None


In [335]:
li = ['state', 1, 'fbc']
ru.master_do( li )

State updated to 1: FBC
 Removing force on base 1. Jack,
 Removing force on base 2. 
 Removing force on base 3. 
 Removing force on base 4. 


TypeError: 'NoneType' object does not support item deletion

In [329]:
ru.print_status()

 Isaac, on 2 occupies None | He is forced to None or must tag-up at None
 Jack, on 0 occupies None | He is forced to 1 or must tag-up at None


In [343]:
"""!!! OTHER STUFF BELOW -- CURRENT STUFF ABOVE !!! """

                        
def fielder_tags_runner(self, li):
    print(f"I think *{li[0]}* is complete")

    name = li[1]
    runner = runners[name]
    if not(runner.occupied_base):
        self.put_out(name)


def fielder_tags_base(self, li):
    print(f"I think *{li[0]}* is complete")
    base = li[1]

    for name, runner in self.runners.items:
        if runner.f2_base == base or runner.tagup_base == base:
            self.put_out(name)
            return


def occupy_attain_base(self, li):
    print(f"I think *{li[0]}* is complete")

    base = li[2]

    ## Check occupied bases
    for name, runner in self.runners:
        if runner.occupied_base == base:
            return

    name = li[1]
    runner = self.runners[name]

    runner.attained_base = runner.occupied_base = base

    ## Clear taggable bases
    if runner.f2_base == base:
        runner.f2_base = None

    if runner.tagup_base == base:
        runner.tagup_base = None


